#Bloom filter

In [ ]:
!pip install bitarray
!pip install mmh3
from bitarray import bitarray 
import mmh3
import hashlib

animals = ['dog', 'cat', 'giraffe', 'fly', 'mosquito', 'horse', 'eagle',
           'bird', 'bison', 'boar', 'butterfly', 'ant', 'anaconda', 'bear',
           'chicken', 'dolphin', 'donkey', 'crow', 'crocodile']

other_animals = ['badger', 'cow', 'pig', 'sheep', 'bee', 'wolf', 'fox',
                 'whale', 'shark', 'fish', 'turkey', 'duck', 'dove',
                 'deer', 'elephant', 'frog', 'falcon', 'goat', 'gorilla',
                 'hawk', 'bison' ]

data = animals+other_animals

     |████████████████████████████████| 236 kB 6.1 MB/s 
     |████████████████████████████████| 50 kB 4.9 MB/s 


In [ ]:
class BloomFilter():
  def __init__(self, size, hash_count):
    self.size = size
    self.hash_count = hash_count
    self.bit_array = bitarray(self.size)
    self.bit_array.setall(0)
    
  def add(self, item):
    for i in range(self.hash_count):
      item = str(item) + str(i)
      index = int.from_bytes(hashlib.md5(item.encode()).digest(), 'big')%self.size  #ity hash jest generowany za pomocą seed-u 'i'
      self.bit_array[index] = 1


  def contain(self, item):
    out = True
    for i in range(self.hash_count):
      item = str(item) + str(i)
      index = int.from_bytes(hashlib.md5(item.encode()).digest(), 'big')%self.size #ity hash jest generowany za pomocą seed-u 'i'
      if self.bit_array[index] == 0:
        out = False
    return out

In [ ]:
def check_animal(data_for_bloom_filter, animal_to_check):
  if bf.contain(animal_to_check) == True and animal_to_check not in data_for_bloom_filter:
    return 1
  else:
    return 0

In [ ]:
def parameters():
  main_loop_range = 20
  sub_loop_range = 20
  bloom_filter_size = 100
  bloom_filter_hash_count = 10
  sample_size_for_data_for_bloom_filter = 20
  sample_size_for_data_to_check = 30
  return main_loop_range, sub_loop_range, bloom_filter_size, bloom_filter_hash_count, sample_size_for_data_for_bloom_filter, sample_size_for_data_to_check


In [ ]:
from random import randint, sample

FNcounter_list = []
FN_counter = 0
[main_loop_range, sub_loop_range, bloom_filter_size, bloom_filter_hash_count, sample_size_for_data_for_bloom_filter, sample_size_for_data_to_check] = [20, 20, 100, 10, 20, 30]

for main_loop in range(main_loop_range): #dajemy losowanie by za kazdym razem było inaczej
  avg_FN_num = 0
  bf = BloomFilter(bloom_filter_size, bloom_filter_hash_count)
  data_for_bloom_filter = sample(data, k = sample_size_for_data_for_bloom_filter)
  [bf.add(element) for element in data_for_bloom_filter]


  for sub_loop in range(sub_loop_range):
    FNcounter = 0
    data_to_check = sample(data, k = sample_size_for_data_to_check)
    for animal_to_check in data_to_check:
     FNcounter += check_animal(data_for_bloom_filter, animal_to_check) 
    avg_FN_num += FNcounter

  avg_FN_num = round(avg_FN_num/sample_size_for_data_to_check)
  FNcounter_list.append((main_loop, avg_FN_num))
  FN_counter += avg_FN_num

print(FNcounter_list)
print(FN_counter/main_loop_range)

[(0, 1), (1, 3), (2, 3), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 3), (10, 5), (11, 1), (12, 2), (13, 4), (14, 3), (15, 4), (16, 2), (17, 2), (18, 2), (19, 3)]
2.5


Conclusions:
For individual loops with the posted parameters, the following conclusions were obtained:
[main_loop_range, sub_loop_range, bloom_filter_size, bloom_filter_hash_count, sample_size_for_data_for_bloom_filter, sample_size_for_data_to_check]
0. [20, 20, 100, 10, 10, 30] Practically no false positives (change in sample_size_for_data_for_bloom_filter count) - initial sample
1. [20, 20, 100, 10, 20, 30] Relatively small number of FN on average 2.05 (change in sample_size_for_data_for_bloom_filter)
2. [20, 20, 100, 10, 10, 10] Changing the value of sample_size_for_data_to_check has no effect on FN,
3. [20, 20, 100, 20, 10, 30] Avg = 0.5-1 Changing the number of hashes affects the average
4. [20, 20, 500, 10, 10, 30] Increasing the size of the binary array (affects FN reduction)



Final conclusions:
- increasing the number of available hashes has a negative effect (an increase in FN is observed)
- increasing the size of the binary array (affects the FN reductions)
- changing the value of sample_size_for_data_to_check has no effect on FN

Translated with www.DeepL.com/Translator (free version)